In [140]:
# Boilerplate at top of file
%load_ext autoreload
%load_ext rmagic
%autoreload 2
import warnings
warnings.filterwarnings('ignore')
import pandas as pd

import numpy as np
import datetime
import math

import matplotlib.pyplot as plt
%matplotlib inline
plt.style.use('ggplot')
from ggplot import *

#####
# custom functions
#####
import etl
from function import *

#####
# custom to script
####
from sklearn import linear_model

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
The rmagic extension is already loaded. To reload it, use:
  %reload_ext rmagic


In [109]:
df_set, df_raw_test = etl.etl('competition_data')
print df_set.shape
Y_KEY = 'cost'

(30213, 53)


In [119]:
from sklearn import preprocessing
enc = preprocessing.LabelEncoder ()
 
df_set['supplier'] = enc.fit_transform(df_set[['supplier']])
df_set['bracket_pricing'] = enc.fit_transform(df_set[['bracket_pricing']])
df_set[feature_keys].head()

,wall,length,diameter,component_count,bracket_pricing,supplier
0,0.71,137,6.35,1,1,36
1,0.71,137,6.35,1,1,36
2,0.71,137,6.35,1,1,36
3,0.71,137,6.35,1,1,36
4,0.71,137,6.35,1,1,36


In [94]:
df_set.columns.values

array(['tube_assembly_id', 'material_id', 'diameter', 'wall', 'length',
       'num_bends', 'bend_radius', 'end_a_1x', 'end_a_2x', 'end_x_1x',
       'end_x_2x', 'end_a', 'end_x', 'num_boss', 'num_bracket', 'other',
       'spec1', 'spec2', 'spec3', 'spec4', 'spec5', 'spec6', 'spec7',
       'spec8', 'spec9', 'spec10', 'component_id_1', 'quantity_1',
       'component_id_2', 'quantity_2', 'component_id_3', 'quantity_3',
       'component_id_4', 'quantity_4', 'component_id_5', 'quantity_5',
       'component_id_6', 'quantity_6', 'component_id_7', 'quantity_7',
       'component_id_8', 'quantity_8', 'end_a_forming', 'end_x_forming',
       'supplier', 'quote_date', 'annual_usage', 'min_order_quantity',
       'bracket_pricing', 'quantity', 'cost', 'component_count',
       'spec_count'], dtype=object)

In [141]:
from sklearn import cross_validation
# Split training and test
df_train, df_test = cross_validation.train_test_split(df_set)
df_train_x = df_train.drop(Y_KEY, 1)
df_train_y = df_train[Y_KEY]
# Testing
df_test_x = df_test.drop(Y_KEY, 1)
df_test_y = df_test[Y_KEY]
df_train_x.head()

,tube_assembly_id,material_id,diameter,wall,length,num_bends,bend_radius,end_a_1x,end_a_2x,end_x_1x,...,end_a_forming,end_x_forming,supplier,quote_date,annual_usage,min_order_quantity,bracket_pricing,quantity,component_count,spec_count
5204,TA-02916,SP-0019,6.35,0.71,93,7,19.05,N,N,N,...,Yes,Yes,36,2013-07-07,0,0,1,50,1,0
11721,TA-06959,SP-0029,88.90,1.65,78,2,101.60,Y,Y,N,...,Yes,Yes,36,2011-08-01,660,40,0,40,2,2
12714,TA-07770,SP-0028,6.35,0.89,215,5,19.05,N,N,N,...,No,No,36,2013-07-28,0,0,1,1,2,0
10786,TA-06343,SP-0019,6.35,0.71,109,2,19.05,N,N,N,...,Yes,Yes,36,2013-06-30,0,0,1,25,1,0
1621,TA-00928,SP-0029,25.40,1.65,143,7,63.50,N,N,N,...,No,Yes,36,2013-10-01,0,0,1,5,1,0


In [229]:
# Features to start with
feature_keys = ['wall', 'length', 'diameter','component_count','bracket_pricing','bend_radius']

# Some Basic Linear Model

In [207]:

df_train_bracketed = df_train[df_train['bracket_pricing'] == 1]
df_train_non_bracketed = df_train[df_train['bracket_pricing'] == 0]
df_test_bracketed = df_test[df_test['bracket_pricing'] == 1]
df_test_non_bracketed = df_test[df_test['bracket_pricing'] == 0]

# Bracketed Models
lm1 = linear_model.LinearRegression()
lm1.fit(df_train_bracketed[feature_keys], df_train_bracketed[Y_KEY])
predict_y = lm1.predict(df_test_bracketed[feature_keys])
predict_y = np.nan_to_num(predict_y)
print metric(predict_y, df_test_bracketed[Y_KEY])


# Non-Bracketed Models
lm2 = linear_model.LinearRegression()
lm2.fit(df_train_non_bracketed[feature_keys], df_train_non_bracketed[Y_KEY])
predict_y = lm2.predict(df_test_non_bracketed[feature_keys])
predict_y = np.nan_to_num(predict_y)
print metric(predict_y, df_test_non_bracketed[Y_KEY])

0.792263705758
nan


# Some Test Random Forest

In [253]:
from sklearn import tree
from sklearn import ensemble

rforest = ensemble.RandomForestRegressor(bootstrap=True)
df_train_x = df_train_x._get_numeric_data()
df_train_x = df_train_x.dropna(axis=1)

df_test_x = df_test_x._get_numeric_data()
df_test_x = df_test_x.dropna(axis=1)


rforest.fit(df_train_x, df_train_y)
predict_y_forest = rforest.predict(df_test_x)

print rforest.feature_importances_
print metric(predict_y_tree, df_test_y)
print metric(predict_y_forest, df_test_y)

[ 0.19278216  0.10635309  0.07667484  0.03773181  0.03618463  0.01160728
  0.00077728  0.00584697  0.06124778  0.17372029  0.03332809  0.00722956
  0.16972255  0.05805029  0.02874337]
0.803523602978
0.300367370665


# R Analysis

In [152]:
%%R -i df_set 
m1 <- lm(cost ~ lspec_count + component_count  
         + spec_count+ quantity + bracket_pricing  
         + wall + bend_radius, data = (df_set))
summary(m1)
# qqnorm(m1.rstand)
# qqline(m1.rstand)


Call:
lm(formula = cost ~ spec_count + component_count + spec_count + 
    quantity + bracket_pricing + wall + bend_radius, data = (df_set))

Residuals:
   Min     1Q Median     3Q    Max 
-79.95  -9.15  -3.28   3.21 945.79 

Coefficients:
                  Estimate Std. Error t value Pr(>|t|)    
(Intercept)      2.6871201  0.6402694   4.197 2.71e-05 ***
spec_count       0.9382378  0.0898960  10.437  < 2e-16 ***
component_count -0.9638331  0.1843943  -5.227 1.73e-07 ***
quantity        -0.0554834  0.0022436 -24.729  < 2e-16 ***
bracket_pricing -2.3409049  0.4854475  -4.822 1.43e-06 ***
wall            11.4830965  0.2561719  44.826  < 2e-16 ***
bend_radius      0.0028783  0.0007842   3.671 0.000242 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Residual standard error: 27.22 on 30206 degrees of freedom
Multiple R-squared:  0.09821,	Adjusted R-squared:  0.09803 
F-statistic: 548.3 on 6 and 30206 DF,  p-value: < 2.2e-16



# Model and Print submission

In [50]:
np_val = mtree.predict(df_raw_test[feature_keys])
df_val = pd.DataFrame(data={'id': list(xrange(1,len(np_val)+1)), 'cost' : np_val})

In [51]:
df_val[['id', 'cost']].to_csv('submission.csv', index=False)